In [2]:
!pip install tflite-runtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 22.4 MB/s eta 0:00:00


In [7]:
%pip install numpy==1.26.4 --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 95.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [1]:
import numpy as np
from PIL import Image
import tflite_runtime.interpreter as tflite

# HAM10000 label map
HAM10000_LABELS = {
    0: "Actinic keratoses (akiec)",
    1: "Basal cell carcinoma (bcc)",
    2: "Benign keratosis-like lesions (bkl)",
    3: "Dermatofibroma (df)",
    4: "Melanoma (mel)",
    5: "Melanocytic nevi (nv)",
    6: "Vascular lesions (vasc)"
}

def load_and_preprocess_image(image_path, target_size=(224, 224)):
    image = Image.open(image_path).convert('RGB')
    image = image.resize(target_size)
    image_array = np.asarray(image, dtype=np.float32) / 255.0  # Normalize
    return np.expand_dims(image_array, axis=0)  # Shape: (1, 224, 224, 3)

def run_skin_disease_model(model_path, image_path, label_map=HAM10000_LABELS):
    # Load and preprocess image
    input_data = load_and_preprocess_image(image_path)

    # Load the TFLite model and allocate tensors
    interpreter = tflite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()

    # Get input and output tensor info
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Set the input tensor
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()

    # Get the output tensor
    output_data = interpreter.get_tensor(output_details[0]['index'])  # Shape: (1, 7)
    predicted_class = int(np.argmax(output_data))
    confidence = float(np.max(output_data))

    return {
        "class_id": predicted_class,
        "label": label_map[predicted_class],
        "confidence": confidence
    }


In [2]:
result = run_skin_disease_model("/content/model.tflite", "/content/Image_Editor.png")
print(f"Prediction: {result['label']}")
print(f"Confidence: {result['confidence']:.2f}")

Prediction: Melanoma (mel)
Confidence: 0.77
